In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import pickle
import numpy as np
import pandas as pd

2025-03-31 17:14:09.439958: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 17:14:09.443234: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 17:14:09.452119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743441249.468896    4738 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743441249.473239    4738 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743441249.484969    4738 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
## Load the trained model , scaler pickle , onehotencoder pickle
model = load_model('model.h5')

# load the encoder and scaler
with open('geo_encoder.pkl', 'rb') as file:
    geo_encoder = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)        


2025-03-31 17:14:17.323145: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Example input data
input_data = {
    'CreditScore': 700,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [4]:
# Ohe the categorical feature 'Geography'
geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
# Combine ohe colums with the rest of the input data
input_data_df = pd.DataFrame([input_data])
input_data_df = input_data_df.join(geo_encoded_df)
# Drop the original 'Geography' column
input_data_df.drop(columns=['Geography'], inplace=True)
input_data_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,Male,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [6]:
# Label encoding the gender column

input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [7]:
# Scale the input data

input_data_scaled = scaler.transform(input_data_df)
input_data_scaled

array([[ 0.50178794,  0.91324755, -0.37056859, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [8]:
## Predict the churn probability

predictions = model.predict(input_data_scaled)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
[[0.01511902]]


In [9]:
prediction_probability = predictions[0][0]

if prediction_probability > 0.5:
    print("Customer is likely to churn.")
else:
    print("Customer is unlikely to churn.")


Customer is unlikely to churn.
